## AMNE 376: A Study of Richter's Kouroi Through Image Clustering

Author: *Kaiyan Zhang, Yash Mali, prAxIs UBC Team*

### 1. Introduction: How Computers See Visual Art?




In [ ]:
# Loading necessary libraries
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import timm
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import umap
import matplotlib.pyplot as plt

### 2. Data Exploration

In [ ]:
# Defining the dataclass for the dataset
class SculptureDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=None):
        self.meta = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.transform = transform or T.Compose([
            T.Resize((224,224)),
            T.ToTensor(),
            T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
        ])

    def __len__(self):
        return len(self.meta)

    def __getitem__(self, idx):
        row = self.meta.iloc[idx]
        img_path = os.path.join(self.img_dir, row['filename'])
        img = Image.open(img_path).convert('RGB')
        x = self.transform(img)
        # optional: return labels for evaluation
        y = row[['group', 'era']].values
        return x, y

### 3. Image Embedding Using Dino V2

### 4. Image Clustering and Component Analysis

### 5. Conclusion

### Reference
1. Richter, G. M. A. (1970). *Kouroi: Archaic Greek youths: A study of the development of the Kouros type in Greek sculpture.* Phaidon. Accessed through Internet Archive <https://archive.org/details/kouroiarchaicgre0000rich>.